# Kaggle Who's comming back for dinner.

Le but de ce projet est de prédire si un client ayant fait une réservation dans un restaurant est susceptible de revenir dans le restaurant.
<br>
Pour cela, nous avons une base de données qui nous renseigne sur les réservations, sur les restaurants et sur les clients.
<br>
Le projet peut être écrit en R ou en python. Le choix de langage est le python

# Sommaire
<a href= "#Importer_les_données"> 1. Importer les données </a>
<br>
<a href= "#Evaluer_les_données"> 2.Évaluer la qualité des données et les valeurs manquantes </a>
<br>
<a href= "#Ajustement"> 3.Ajustement des données </a>
<p style="text-indent:20px;">  <a href= "#Ajustement_datetime"> 3.1.Ajustement de la colonne datetime </a>  </p>
<p style="text-indent:20px;">  <a href= "#Ajustement_status"> 3.2.Ajustement de la colonne status </a>  </p>
<p style="text-indent:20px;">  <a href= "#Ajustement_purpose"> 3.3.Ajustement de la colonne purpose </a>  </p>
<p style="text-indent:20px;">  <a href= "#Ajustement_autres"> 3.3.Ajustement d'autres colonnes</a>  </p>
<a href= "#Ajout_restaurant"> 4. Ajout de nouvelles données </a>
<br>
<a href= "#preparation">  5.Préparation des données </a>
<br>
<a href= "#Execution">  6.Exécution de simples algorithmes de régression linéaire </a>
<br>
<a href= "#AUC">  7. Théorie de la courbe ROC </a>
<br>
<a href= "#randomclassifier">  8. Algorithme RandomClassifier </a>
<br>
<a href= "#Logistic">  9. Algorithme régression logistique </a>
<br>
<a href= "#boost">  10. Algorithme régression logistique et boosting  </a>
<br>






Voici la liste des packages susceptibles d'être utilisés dans un projet de Deep Learning. Nous pouvons mettre en lumière deux bibliothèques les plus essentielles: 
    - Numpy qui crée une Data Frame
    - Scikit-learn qui sert notamment à l'apprentissage automatique des données en fonction d'algorithmes mathématiques 

In [ ]:
# Main packages
import numpy as np
import pandas as pd
import xgboost as xgb

# Graph package 
import matplotlib.pyplot as plt
import seaborn as sns


# Classifiers
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
#from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier
#from sklearn.feature_selection import SelectKBest ,chi2,RFE


# Tools
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score



#Yes pourquoi pas 
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score


#Noramlize your Data bro
from sklearn.preprocessing import StandardScaler



# <p id="Importer_les_données"> 1.Importer les données </p>

Dans un premier temps, nous exportons tous les datas .csv en data frame avec la bibliothèque numpy



In [ ]:
# Importing data
test = pd.read_csv('../input/data-kaggle/Test.csv')
train = pd.read_csv('../input/data-kaggle/Train.csv')

#Add extra data on your Data Set 
restaurant = pd.read_csv('../input/data-kaggle/restaurant.csv', sep=';', encoding='utf8')
#members = pd.read_csv('Data/member.csv', sep=';', encoding='utf8')




Nous travaillons dans un premier temps avec deux data frame, Train et Test

In [ ]:
#preview test data
test.head()

In [ ]:
print('Le nombre de ligne dans le tableau test est :{}.'.format(test.shape[0]))

Note :  il n'y a pas de variable cible dans les données de test (c'est-à-dire que la colonne "Return90" est manquante), l'objectif est donc de prévoir cette cible en utilisant différents algorithmes d'apprentissage.



# <p id="Evaluer_les_données"> 2.Évaluer la qualité des données et les valeurs manquantes </p>

Quand on se confronte pour la première fois à un projet de machine learning, on peut être tenté de croire que la problématique principale s’articule autour de l’algorithme. S’il est vrai que cette partie doit être parfaitement maîtrisée pour permettre aux prédictions d’atteindre des résultats d’une fiabilité suffisamment élevée pour être exploitables, le choix et le paramétrage des algorithmes ne représentent que peu souvent la problématique la plus complexe. Le résultat optimal peut être atteint seulement avec un bon nettoyage des données.

Les valeurs nulles trompent les algorithmes de prédictions et faussent le résultat. Il faut donc tout d'abord les traiter et les remplacer.
<br>
Nous remarquons que les valeurs nulles sont trouvées dans la colonne "purpose"


In [ ]:
print('Train\n',train.isnull().sum())
print('.................')
print('.................')
print('Test\n',test.isnull().sum())

In [ ]:
train['purpose'].value_counts().plot(kind='barh', figsize=(6,6))


In [ ]:
print('Le pourcentage de valeurs nulles dans la colonne "purpose"  est  %.2f%% ' %((train['purpose'].isnull().sum()/train.shape[0])*100))

Nous définissons alors que si un client n'a pas rempli la colonne 'purpose", alors il a fait une réservation sans aucune intention particulière.
<br>
Donc toutes les valleurs nulles sont remplacées par le but 'Aucun'

In [ ]:
def fill_na(df): 
    df['purpose'] = df['purpose'].fillna('Aucun')

# <p id="Ajustement"> 3. Ajustement des données </p>

La data base fournie des données brutes, il faut donc les traiter pour leur donner du sens ou bien ajouter des paramètres à l'algorithme.
<br>
Nous traiterons alors les colonnes : 'purpose', 'datetime', 'gender', 'status' et nous supprimons la colonne 'cdate' qui n'apporte aucune information  à l'algorithme  de prédiction





# <p id="Ajustement_datetime"> 3.1. Ajustement de la colonne datetime </p>


In [ ]:
train["datetime"].head(2)

La colonne datetime nous présente des données en timestamp, inconvénient pour un algorithme de prédiction. Cependant,  nous pouvons en extraire le mois et l'heure. 
<br>
La bibliothèque 'pandas permet d'extraire ces informations avec la fonction 'DatetimeIndex'.
<br>
Nous calculons l'heure de la réseravtion(diner, déjeuner), le mois et aussi le nombre de jours d'avance que le client à réservé. 

In [ ]:
#Clean la colone datetime en 4 colonnes, month, heure du déjeuner, heure du souper, nombre de jours réservation avant 
def date(df):
    df['d1'] = pd.to_datetime(df['datetime'])
    df['month']=pd.DatetimeIndex(df['d1']).month
    df['dejeuner']= pd.DatetimeIndex(df['d1']).hour< 17
   
    df['diner']= (pd.DatetimeIndex(df['d1']).hour >=17)

    df['nb_jours_avance'] = ((df["d1"]) - pd.to_datetime(df['cdate_x'])).dt.days
    
    return df

# <p id="Ajustement_status"> 3.2. Ajustement de la colonne status </p>


La colonne "status" nous donne des indications sur le statut de la réservation; la variable peut prendre plusieurs valeurs : 'changed', 'no-show', 'new','canceled', 'ok'.


In [ ]:
train['status'].value_counts().plot(kind='barh', figsize=(5,5))

Nous utilisons les 'dummy', cette fonction permet de convertir une colonne et ses X variables, en X nouvelles colonnes booléennes dont les titres sont les variables 

In [ ]:
pd.get_dummies( train['status']).head(5)

Initialement, nous voulons créer seulement deux colonnes, si le client se présente ou ne se présente pas.
<br>
Mais avec un peu plus de recul, l'information brute apporte plus d'informations à l'algorithme.
<br>
Finalement, la solution finale est juste un dummy de la colonne statut.

In [ ]:
def  status(df):

    dummy = pd.get_dummies( df['status'])
    #dummy ['present'] = dummy['changed']+dummy['ok']+ dummy['new']
    #dummy['absent'] = dummy['no-show']+dummy['canceled']

    #dummy.drop('canceled',inplace=True, axis=1)
    #dummy.drop('changed',inplace=True, axis=1)
    #dummy.drop('new',inplace=True, axis=1)
    #dummy.drop('no-show',inplace=True, axis=1)
    #dummy.drop('ok',inplace=True, axis=1)
    
    return dummy




# <p id="Ajustement_purpose"> 3.3. Ajustement de la colonne purpose</p>


In [ ]:
test['purpose'].value_counts().head(20).plot(kind='barh')


L'ajustement de la colonne "purpose" nous a donné du fil à retordre. La première solution apportée est de sélectionner seulement les variables utiles. En effet, après visualisation du graphe ci-dessus, il y a seulement 6 variables interférentes sur 16. Après les avoir sélectionnées il suffit de l'insérer dans notre tableau final (voir fonction ci-dessus). Cette fonction n'a pas été retenue, puisque lors du rendu final du projet, le site Kaggle exigeait le nombre initial de lignes. Ainsi, nous avons compris que nous ne pouvons pas supprimer de lignes 

In [ ]:
def purpose(df):
    df.drop((df[~df['purpose'].str.contains('anniversaire')].index) & 
            (df[~df['purpose'].str.contains('amis')].index) & 
            (df[~df['purpose'].str.contains('famille')].index) & 
            (df[~df['purpose'].str.contains('Sweet')].index) & 
            (df[~df['purpose'].str.contains('affaires')].index) & 
            (df[~df['purpose'].str.contains('importante')].index) & 
            (df[~df['purpose'].str.contains('Aucun')].index), inplace = True)

    return df

La solution retenue est d'utiliser les "dummy".Ainsi, créer un tableau temporaire booléen, et de les réunir par sujet (anniversaire, amis, famille....)

In [ ]:
def dummy_purpose_train(df):
    dummy = pd.get_dummies(df['purpose'])
    df['anniversaire']= (dummy['Birthday']== 1) | (dummy['Birthday,Celebration']== 1 ) | (dummy["Célébration d'anniversaire"]==1) | (dummy["Fête d'anniversaire"]==1)
    df['amis']=(dummy['Friends,Reunion']== 1) | (dummy['Dîner entre amis']== 1 )
    df['famille'] = (dummy['Family,Gathering']== 1) | (dummy['Repas en famille']== 1 )| (dummy['Repas en famille,盡量靠W']== 1 )| (dummy['Repas en famille-生日']== 1 ) 
    df['business'] = (dummy["Dîner d'affaires" ]== 1)
    df['Date_tinder'] = (dummy['Sweet day' ]== 1)
    df['importante'] = (dummy['Date importante' ]== 1)
    df['autre']=(dummy['Aucun']== 1) |(dummy['Veuillez sélectionner']== 1) | (dummy['其他']== 1) |(dummy['Please,Select']== 1) | (dummy['&#65533;&#40115;&#65533;&#65533;&#26813;&#65533;']== 1) | (dummy['家人&#65533;']== 1)
    return df


In [ ]:
def dummy_purpose_test(df):
    dummy = pd.get_dummies(df['purpose'])
    df['anniversaire']= (dummy["Fête d'anniversaire"]== 1) | (dummy["Célébration d'anniversaire"]== 1)
    df['amis'] = (dummy["Dîner entre amis"]== 1) | (dummy["Friends,Reunion"]== 1)| (dummy["Friends"]== 1)
    df['famille'] = (dummy["Repas en famille"]== 1) | (dummy["Family,Gathering"]== 1)
    df['business'] = (dummy["Dîner d'affaires" ]== 1)| (dummy["Business,meeting"]== 1)
    df['Date_tinder'] =(dummy['Sweet day' ]== 1)
    df['importante'] = (dummy['Date importante' ]== 1)
    df['autre']=(dummy['Aucun']== 1) |(dummy['Veuillez sélectionner']== 1) | (dummy['其他']== 1) |(dummy['Please,Select']== 1) |(dummy['慶生']== 1) | (dummy["&#65533;&#40115;&#65533;&#65533;&#26813;&#65533;"]== 1)
    return df


Cependant, cette partie du code pose des problèmes dans la suite du projet et dans la maintenance. En effet, si les restaurants ajoutent d'autres variables, alors le programme ne sera pas en mesure de les prendre en compte.

# <p id="Ajustement_autres"> 3.4. Ajustement des autres colonnes</p>


Nous avons aussi modifier de simple colone pour les rendre bouléennes.

In [ ]:
def gender(df):
    dummy = pd.get_dummies( df['gender'])
    df['Femme']=(dummy['F']==1)
    return df


In [ ]:
def join(df1,df2):
    df3  = df1.join(df2, how='inner')
    return df3

In [ ]:
def present(df):
    df['Present']= (df['present']==1)
    return df

# <p id="Ajout_restaurant"> 4.Ajout de nouvelles données</p>

Afin d'avoir des résultats optimaux, nous avons rajouté des informations à nos bases de données. Le tableau "restaurant" qui apporte des indications sur les restaurants. 
La fusion des deux tableaux se fait sur la clé unique 'restaurant_id' et l 'id'. 

In [ ]:
# Add csv Restaurant on your Train and Test data set
resultat_train = pd.merge(train, restaurant, left_on= 'restaurant_id', right_on='id',how='left')
resultat_test = pd.merge(test, restaurant, left_on= 'restaurant_id', right_on='id',how='left')

resultat_test=resultat_test.set_index('booking_id')
resultat_train=resultat_train.set_index('booking_id')

En ajoutant de nouvelles données, il est nécessaire de recommencer toutes les étapes de nettoyage. 
<br>
Enlever toutes les valeurs nulles, identifier les valeurs utiles et faire des ajustements si nécessaire

In [ ]:
resultat_train.isnull().sum()


Après avoir identifié les colonnes qui contiennent des variables nulles, il faut les remplacer par des valeurs logiques.

Les variables  'is_hotel', 'good_for_family', 'accept_credit_card', 'parking', 'outdoor_seating', 'wifi', 'wheelchair_accessible' sont remplacées par 0 si elles ne sont pas indiquées.

Les variables 'price1 ' et 'price2' sont remplacées par une moyenne des deux. 

In [ ]:
def fill_resultat(df):
    df['is_hotel'] = df['is_hotel'].fillna(0)
    df['good_for_family'] = df['good_for_family'].fillna(0)
    df['accept_credit_card'] = df['accept_credit_card'].fillna(0)
    df['parking'] = df['parking'].fillna(0)
    df['outdoor_seating'] = df['outdoor_seating'].fillna(0)
    df['wifi'] = df['wifi'].fillna(0)
    df['wheelchair_accessible'] = df['wheelchair_accessible'].fillna(0)


    df['price1'] = df['price1'].fillna(df.price1.mean())
    df['price2'] = df['price2'].fillna(df.price2.mean())
    df['cdate_y'] = df['cdate_y'].fillna('Aucun')


    df['id'] = df['id'].fillna('Pasid')
    
    
    df['purpose'] = df['purpose'].fillna('Aucun')
    
    return df


Toutes les informations ci-dessous sont utiles pour  l'algorithme de prédiction. Cependant, si nous lançons cet algorithme avec ces données le résultat ne sera pas optimal.

In [ ]:
restaurant.head(5)

Nous créons deux nouvelles colonnes :
  - accessible qui comprend les variables 'parking' et 'wheelchair_accesible'
  - agréable qui comprend les variables 'accept_credit_card', 'wifi' et 'outdoor_seeting'
    
    
Nous créons aussi un prix moyen de la carte du restaurant. Nous catégorisons si ce restaurant est luxueux, normal ou accessible (avec les paramètres 'prix_haut', 'prix_moyen', 'prix_bas' respectivement)


Nous prenons aussi en compte la date du restaurant pour déterminer s'il est vieux ou jeune. Un restaurant est défini comme vieux lorsqu'il a plus de 5 ans.

In [ ]:
def modif_restaurant(df):

    df['accesible']= df["parking"] + df["wheelchair_accessible"]
    df['agréable']= df["accept_credit_card"] + df["wifi"]+ df["outdoor_seating"]


    df['prix'] = (df['price1']+df['price2'])/2
    
    
    df['prix_haut']=df['prix']>=800
    df['prix_moyen']=(df['prix']>=488) & (df['prix']<800)
    df['prix_bas']=df['prix']<488
    
    
    df["date"] = 2020 - (pd.to_datetime(df["cdate_y"],errors='coerce')).dt.year
    
    df['date'] = df['date'].fillna(df.date.mean())
    
    df['vieux_resto']=df['date']>=5
    df['jeune_resto']=df['date']<5
    return df




# <p id="preparation"> 5. Préparation des données</p>

Enfin nous exécutons les commandes pour la base Train et la base Test et nous supprimons les variables inutiles.

In [ ]:
def prepare_data(df):
    
    gender(df)
    
    
    df4 = status(df)
    df5 = join(df,df4)
    
    df6 = date(df5)
    df7=modif_restaurant(df6)
    
    
    colums = ['member_id', 'cdate_x', 'restaurant_id', 'datetime','purpose','gender',
              'status','is_required_prepay_satisfied', 'd1','cdate_y','id',
              'parking','wheelchair_accessible','accept_credit_card','wifi','outdoor_seating',
              'price1','price2','prix','date']
    
    df7.drop(colums,inplace=True, axis=1)
    
    return df7

In [ ]:
fill_resultat(resultat_train)
fill_resultat(resultat_test)


test = dummy_purpose_test(resultat_test)
train = dummy_purpose_train(resultat_train)

df_test=prepare_data(test)
df_train=prepare_data(train)


Voici le résultat final de la base Test:

In [ ]:
df_test.head()

Avant de passer aux algorithmes nous séparons notre base en 2/3 d'apprentissage et en 1/3 de test.

In [ ]:
#creer une base X_train y_train et X_test et y_test (divise en 2/3 apprentissage, 1/3 test)
X_all = df_train.drop(['return90'], axis=1)
y_all = df_train['return90']

num_test = 0.33
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=2)




 # <p id="Execution"> 6.Exécution de simple algorithme de regression linéaire</p>

Pour commencer nous exécutons un algorithme de régression linéaire.

La régression linéaire est un algorithme de machine learning qui consiste à trouver la meilleure fonction permettant de définir une variable de sortie (l’élément à prédire) à partir d’une  seule et unique variable explicative en entrée (le prédicteur).Graphiquement, il s’agit de trouver la meilleure droite possible pouvant expliquer un modèle (x, y). Cette fonction est de la forme Y = BO + XB1 + E, où Y est la réponse, BO l’ordonnée à l’origine, X la variable explicative, B1 la pente (la valeur d’évolution de X lorsqu’il augmente d’une unité), et enfin E, l’erreur statistique.

Vous trouvez ci-dessous, un algorithme simple. Cependant, le Kaggle ne calcule pas le score selon une simple droite, mais selon l'aire sous la courbe ROC

In [ ]:
# build
linreg = LinearRegression()

# train
linreg.fit(X_train, y_train)

# predict
y1_pred = linreg.predict(df_test)



 # <p id="AUC"> 7. Théorie de la courbe ROC</p>

Le Kaggle détermine le score en fonction de l'aire sur la courbe ROC.

Une courbe ROC (receiver operating characteristic) est un graphique représentant les performances d'un modèle de classification. Cette courbe trace le taux de vrais positifs en fonction du taux de faux positifs :


<img src="https://developers.google.com/machine-learning/crash-course/images/ROCCurve.svg" height="50%" width="50%">

# AUC : aire sous la courbe ROC
<br>
AUC signifie "aire sous la courbe ROC". Cette valeur mesure l'intégralité de l'aire sous la courbe ROC .

<img src="https://developers.google.com/machine-learning/crash-course/images/AUC.svg" height="50%" width="50%">

L'AUC fournit une mesure  des performances pour tous les seuils de classifications possibles. On peut interpréter l'AUC comme une mesure de la probabilité pour que le modèle classe un exemple positif aléatoire au-dessus d'un exemple négatif aléatoire. 

 # <p id="randomclassifier"> 8.Algortithme Random Classifier</p>


Les forêts d'arbres décisionnels (ou random forest classifier) font partie des techniques d'apprentissage automatique. Cet algorithme combine les concepts de sous-espaces aléatoires et de bagging. L'algorithme random forest effectue un apprentissage sur de multiples arbres de décision entraînés sur des sous-ensembles de données légèrement différents.

Les paramètres de l'algorithme sont disponibles sur la documentation de Scikit-learn. Cependant pour choisir les bons paramètres, il faut les sélectionner par dichotomie. Par exemple, le paramètre: 
<br>
-->'n_estimators': [ 300,400,500]

L'algorithme exécute avec les 3 possibilités, et nous retourne la valeur idéale pour nos données (ici n_estimators=300). Ainsi au fur et à mesure, nous trouvons les paramètres idéaux pour l'algorithme.

In [ ]:
# Choose the type of classifier. 
clf = RandomForestClassifier()

# Choose some parameter combinations to try
parameters = {'n_estimators': [ 300,400,500], 
              'max_features': ['auto' ], 
              'criterion': ['gini', 'entropy'],
              'max_depth': [10],
              'min_samples_split': [500],
              'min_samples_leaf': [2]
             }



# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(roc_auc_score)

# Run the grid search
grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer, n_jobs=-1)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_
print(clf)

# Fit the best algorithm to the data. 
best_clf = clf.fit(X_train, y_train)
print('Train score', clf.score(X_train, y_train))
print('Test score', clf.score(X_test, y_test))


#plus proche voisin,  svm
#regarder la mtrice de confusion avec train test split
#AUC detection 

In [ ]:
y_pred = best_clf.predict_proba(X_test)[:,1]


fpr, tpr, _ = roc_curve(y_test,y_pred)
plt.plot(fpr, tpr, marker='.', label='LogistiIIc')


print("Le score AUC est: {} ".format(roc_auc_score(y_test, y_pred)))

 # <p id="Logistic"> 9.Algorithme Régression Logistique </p>


Nous fonctionnons avec la même logique pour la régression logistique 

In [ ]:
regl = LogisticRegression()

parameters = {'penalty': ['l2'],
              'max_iter': [1000],
              'C': [3.0],
              'solver' :[ 'newton-cg','sag', 'lbfgs'],
              'multi_class':['auto', 'ovr', 'multinomial']
             }


# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(roc_auc_score)




# Run the grid search
grid_obj = GridSearchCV(regl, parameters, scoring=acc_scorer, n_jobs=-1, cv=None)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
regl = grid_obj.best_estimator_
print(regl)

# Fit the best algorithm to the data. 

regl.fit(X_train, y_train)
print('Train score', regl.score(X_train, y_train))
print('Test score', regl.score(X_test, y_test))




In [ ]:
y1_pred = regl.predict_proba(X_test)[:,1]


fpr, tpr, _ = roc_curve(y_test,y_pred)
plt.plot(fpr, tpr, marker='.', label='LogistiIIc')


print("Le score AUC est: {}".format(roc_auc_score(y_test, y1_pred)))

 # <p id="boost"> 10.Algorithme Régression logistique et boosting  </p>


La méthode du gradient boosting sert à renforcer un modèle qui produit des prédictions faibles.
<br>
Ainsi nous choisissons aussi les paramètres du gradient boosting qui nous propose des paramètres idéaux pour la régression linéaire

In [ ]:
xgb_model = XGBClassifier()



parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'booster':['gbtree'],
              'learning_rate': [0.02], #so called eta value
              'max_depth': [18],
              'min_child_weight': [13],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.5],
              'n_estimators': [100], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}

xgb= RandomizedSearchCV(estimator=xgb_model, param_distributions=parameters, n_jobs=-1, cv=3, verbose=2, random_state=1,n_iter=7,scoring='roc_auc')


#xgb = GridSearchCV(xgb_model, parameters,scoring='roc_auc',n_jobs=-1)


xgb.fit(X_train, y_train)

xgb.best_params_







Ci-dessous, les paramètres idéaux de la régression linéaire.

In [ ]:
xgb = grid_obj.best_params_
print(xgb)

In [ ]:
estimators= LogisticRegression(C=3.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=2,
                   warm_start=False)





xgb_clas=XGBClassifier(estimator=estimators, nthread=4,booster='gbtree',learning_rate=0.01,max_depth=14,
                      min_child_weight=11,silent=1,subsample=0.5,colsample_bytree=0.5,
                      n_estimators=500,missing=-999,seed=1337)

# Fit the best algorithm to the data. 

xgb_clas.fit(X_train, y_train)
print('Train score', xgb_clas.score(X_train, y_train))
print('Test score', xgb_clas.score(X_test, y_test))


In [ ]:
y2_pred = xgb_clas.predict_proba(X_test)[:,1]


fpr, tpr, _ = roc_curve(y_test,y_pred)
plt.plot(fpr, tpr, marker='.', label='LogistiIIc')


print("Le score AUC est: {} ".format(roc_auc_score(y_test, y2_pred)))

 # <p id="fin"> 11. Génération des résultats </p>


In [ ]:
resultat = xgb_clas.predict_proba(df_test)[:, 1]
print(resultat)

In [ ]:
df_res = pd.DataFrame({'booking_id':test.index, 'return90':resultat})
df_res.to_csv('resultat_hugo.csv', index=False)
resu = pd.read_csv('resultat_hugo.csv', index_col='booking_id')
resu